In [9]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor 
import matplotlib.pyplot as plt
import os.path
from os import path
from sklearn.neural_network import MLPRegressor

In [3]:
def LoadRawVariables():
    c2pt = []
    ts   = []
    taus = []
    xs   = []
    ys   = []
    zs   = []
    c3pt = []
    
    c2pt_start = 9
    c3pt_start = 10155
    
    for tau in range(0, 49, 8):
        for x in range(0, 25, 8):
            for y in range(0, 25, 8):
                for z in range(0, 25, 8):
                    for sample in range(748, 1421, 16):
                        fname = "../Data/T" + str(tau) + "/x" + str(x) + "y" + str(y) + "z" + str(z) + "/nuc3pt.dat." + str(sample)
                        if path.exists(fname):
                            with open(fname) as fp:
                                for i, line in enumerate(fp):
                                    if i >= 7 and i <= 70:
                                        c2pt.append([float(x) for x in line.rstrip().split()[1:3]])
                                        ts.append(i - 7)
                                        taus.append(tau)
                                        xs.append(x)
                                        ys.append(y)
                                        zs.append(z)
                                    elif i >= 10154 and i <= 10217:
                                        c3pt.append([float(x) for x in line.rstrip().split()[1:3]])
                                    elif i > 10217:
                                        break
    
    return ts, taus, xs, ys, zs, c2pt, c3pt

ts, taus, xs, ys, zs, c2pt, c3pt = LoadRawVariables()

## Prediction of c3pt based on c2pt directly

In [4]:
features = np.array([np.array([ts[i], taus[i], xs[i], ys[i], zs[i], c2pt[i][0], c2pt[i][1]]) for i in range(len(ts))])
labels = np.array([np.array([c3pt[i][0], c3pt[i][1]]) for i in range(len(c3pt))])

In [5]:
labelFrac = 0.9
BCFrac = 0.05

labelEnd = int(len(labels) * labelFrac)
BCEnd    = int(len(labels) * (BCFrac + labelFrac))

X_train, Y_train, X_bc, Y_bc, X_test, Y_test = features[:labelEnd], labels[:labelEnd], features[labelEnd:BCEnd], labels[labelEnd:BCEnd], features[BCEnd:], labels[BCEnd:]

gbr_real = GradientBoostingRegressor(learning_rate=0.05, n_estimators=20, max_depth=3)
gbr_real.fit(X_train, Y_train[:, 0])

y_bc_pred_real = gbr_real.predict(X_bc)

biasCrxn_real = np.average(Y_bc[:, 0] - y_bc_pred_real)

gbr_imaginary = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, max_depth=3)
gbr_imaginary.fit(X_train, Y_train[:, 1])

y_bc_pred_imaginary = gbr_imaginary.predict(X_bc)

biasCrxn_imaginary = np.average(Y_bc[:, 1] - y_bc_pred_imaginary)

In [6]:
print(biasCrxn_real)
print(biasCrxn_imaginary)

-48577280898.236565
-69320103552.21849


In [7]:
trials = 0
error = 0
raw_RMS = 0
for i in range(len(X_test)):
    testImg = X_test[i]
    testLabel = Y_test[i]
    pred_real = gbr_real.predict([testImg]) + biasCrxn_real
    pred_imaginary = gbr_imaginary.predict([testImg]) + biasCrxn_imaginary
    error += np.sqrt((pred_real - testLabel[0]) ** 2 + (pred_imaginary - testLabel[1]) ** 2)
    raw_RMS += np.sqrt((testLabel[0]) ** 2 + (testLabel[1]) ** 2)
    trials += 1
error /= trials
print(error)
print([raw_RMS])

[8.75286102e+11]
[9275451977509180.0]


## Prediction of c3pt conglomerate based on c2pt snapshot

In [13]:
# features = np.array([np.array([ts[i], taus[i], xs[i], ys[i], zs[i], c2pt[i][0], c2pt[i][1]]) for i in range(len(ts))])
features_unshifted = np.array([[taus[i]] + [c2pt[i + j][0] for j in range(64)] + [c2pt[i + j][1] for j in range(64)] for i in range(0, len(ts), 64)])
features = []
for f in features_unshifted:
    shift = int(f[0])
    features.append(np.roll(f[1:], -shift))

#### THIS IS WORSE ---> Try sparsely sampling features to combat large amount of data 
# for i in range(len(features)):
#     sparseSubFeature = np.array([features[i][j] for j in range(0, len(features[i]), 8)])
#     features[i] = sparseSubFeature

features = np.array(features)

labels = np.array([sum(c3pt[i:i+64][0]) / 64 for i in range(0, len(c3pt), 64)])

print(len(features), len(labels))
print(len(features[0]))

4368 4368
128


In [14]:
labelFrac = 0.9
BCFrac = 0.05

labelEnd = int(len(labels) * labelFrac)
BCEnd    = int(len(labels) * (BCFrac + labelFrac))

X_train, Y_train, X_bc, Y_bc, X_test, Y_test = features[:labelEnd], labels[:labelEnd], features[labelEnd:BCEnd], labels[labelEnd:BCEnd], features[BCEnd:], labels[BCEnd:]

gbr = GradientBoostingRegressor(learning_rate=0.05, n_estimators=50, max_depth=3)
gbr.fit(X_train, Y_train)

y_bc_pred = gbr.predict(X_bc)

biasCrxn = np.average(Y_bc - y_bc_pred)

In [15]:
trials = 0
error = 0
raw_RMS = 0
for i in range(len(X_test)):
    testImg = X_test[i]
    testLabel = Y_test[i]
    pred = gbr.predict([testImg]) + biasCrxn
    error += abs(pred - testLabel)
    trials += 1
    
error /= trials
print(error / sum(abs(Y_test)) * len(Y_test))

[0.9797405]


## Train based on a NN rather than a Decision Tree

In [30]:
# features = np.array([np.array([ts[i], taus[i], xs[i], ys[i], zs[i], c2pt[i][0], c2pt[i][1]]) for i in range(len(ts))])
features_unshifted = np.array([[taus[i]] + [c2pt[i + j][0] for j in range(64)] + [c2pt[i + j][1] for j in range(64)] for i in range(0, len(ts), 64)])
features = []
for f in features_unshifted:
    shift = int(f[0])
    features.append(np.roll(f[1:], -shift))

#### Try sparsely sampling features to combat large amount of data 
# for i in range(len(features)):
#     sparseSubFeature = np.array([features[i][j] for j in range(0, len(features[i]), 8)])
#     features[i] = sparseSubFeature

features = np.array(features)

labels = np.array([sum(c3pt[i:i+64][0]) / 64 for i in range(0, len(c3pt), 64)])

print(len(features), len(labels))
print(len(features[0]))

4368 4368
128


In [33]:
labelFrac = 0.9

labelEnd = int(len(labels) * labelFrac)

X_train, Y_train, X_test, Y_test = features[:labelEnd], labels[:labelEnd], features[labelEnd:], labels[labelEnd:]

mlp = MLPRegressor(max_iter=1000, learning_rate="adaptive", learning_rate_init=0.05)
mlp.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='adaptive',
             learning_rate_init=0.05, max_fun=15000, max_iter=1000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [34]:
trials = 0
error = 0
raw_RMS = 0
for i in range(len(X_test)):
    testImg = X_test[i]
    testLabel = Y_test[i]
    pred = mlp.predict([testImg])
    error += abs(pred - testLabel)
    trials += 1
    
error /= trials
print(error / sum(abs(Y_test)) * len(Y_test))

[0.99228135]
